In [ ]:
# !pip install -U s3fs

In [1]:
from rich import print

## Pangeo Forge: Transforming archival data into analysis-ready, cloud-optimized (ARCO) data stores

[NSF Award 2026932](https://www.nsf.gov/awardsearch/showAward?AWD_ID=2026932)

_Frontiers In Climate_ paper: https://doi.org/10.3389/fclim.2021.782909

Slides: https://github.com/cisaacstern/pangeo-forge-slides/blob/esip-feb-2022/slides.ipynb

## NOAA Optimum Interpolation Sea Surface Temperature (OISST) `http` index
https://www.ncei.noaa.gov/data/sea-surface-temperature-optimum-interpolation/v2.1/access/avhrr/
```
 │
 ├──198109/
 │   ├──oisst-avhrr-v02r01.19810901.nc
 │  ...
 │   └──oisst-avhrr-v02r01.19810930.nc
...
 └──202110/
```

In [2]:
!wget 'https://www.ncei.noaa.gov/data/sea-surface-temperature-optimum-interpolation/v2.1/access/avhrr/198109/oisst-avhrr-v02r01.19810901.nc'

--2022-02-28 09:46:55--  https://www.ncei.noaa.gov/data/sea-surface-temperature-optimum-interpolation/v2.1/access/avhrr/198109/oisst-avhrr-v02r01.19810901.nc
Resolving www.ncei.noaa.gov... 2610:20:8040:2::177, 2610:20:8040:2::172, 2610:20:8040:2::168, ...
Connecting to www.ncei.noaa.gov|2610:20:8040:2::177|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1714749 (1.6M) [application/x-netcdf]
Saving to: 'oisst-avhrr-v02r01.19810901.nc.17'

oisst-avhrr-v02r01. 100%[===================>]   1.63M  2.74MB/s    in 0.6s    

2022-02-28 09:46:57 (2.74 MB/s) - 'oisst-avhrr-v02r01.19810901.nc.17' saved [1714749/1714749]



In [3]:
import xarray as xr

ds = xr.open_dataset("oisst-avhrr-v02r01.19810901.nc")
nbytes = ds.nbytes
print(f"{nbytes/1e6:.2f} MB")
print(ds)

16.60 MB

<xarray.Dataset>
Dimensions:  (time: 1, zlev: 1, lat: 720, lon: 1440)
Coordinates:
  * lat      (lat) float32 -89.88 -89.62 -89.38 -89.12 ... 89.38 89.62 89.88
  * lon      (lon) float32 0.125 0.375 0.625 0.875 ... 359.1 359.4 359.6 359.9
  * time     (time) datetime64[ns] 1981-09-01T12:00:00
  * zlev     (zlev) float32 0.0
Data variables:
    anom     (time, zlev, lat, lon) float32 ...
    err      (time, zlev, lat, lon) float32 ...
    ice      (time, zlev, lat, lon) float32 ...
    sst      (time, zlev, lat, lon) float32 ...
Attributes: (12/37)
    title:                      NOAA/NCEI 1/4 Degree Daily Optimum Interpolat...
    source:                     ICOADS, NCEP_GTS, GSFC_ICE, NCEP_ICE, Pathfin...
    id:                         oisst-avhrr-v02r01.19810901.nc
    naming_authority:           gov.noaa.ncei
    summary:                    NOAAs 1/4-degree Daily Optimum Interpolation ...
    cdm_data_type:              Grid
    ...                         ...
    metadata_link:              https://doi.org/10.25921/RE9P-PT57
    ncei_template_version:      NCEI_NetCDF_Grid_Template_v2.0
    comment:                    Data was converted from NetCDF-3 to NetCDF-4 ...
    sensor:                     Thermometer, AVHRR
    Conventions:                CF-1.6, ACDD-1.3
    references:                 Reynolds, et al.(2007) Daily High-Resolution-...

In [4]:
import pandas as pd

# Our target range includes 14372 files
dates = pd.date_range("1981-09-01", "2021-01-05", freq="D")

print(f"{len(dates)} files -> {(len(dates)*nbytes)/1e9:.2f} GB")

14372 files -> 238.54 GB

## The old way: manual Zarr conversion

- Reproducibility (i.e. provenance chain)
- Scalability
- Maintainability


# A better way: Pangeo Forge

1. `pangeo-forge-recipes`: Encodes complete ARCO data transformation pipeline as code which can be version controlled.
2. **Pangeo Forge Cloud**: Automatation of recipe execution in the cloud. Integrated with GitHub.

In [7]:
# https://github.com/pangeo-forge/staged-recipes/blob/master/recipes/noaa-oisst/recipe.py

from noaa_oisst_recipe import recipe

for i, (index, url) in enumerate(recipe.file_pattern.items()):
    if i < 2 or i > 14369:
        print(repr(index), url)

Index({DimIndex(name='time', index=0, sequence_len=14372, operation=<CombineOp.CONCAT: 2>)}) 
https://www.ncei.noaa.gov/data/sea-surface-temperature-optimum-interpolation/v2.1/access/avhr
r/198109/oisst-avhrr-v02r01.19810901.nc

Index({DimIndex(name='time', index=1, sequence_len=14372, operation=<CombineOp.CONCAT: 2>)}) 
https://www.ncei.noaa.gov/data/sea-surface-temperature-optimum-interpolation/v2.1/access/avhr
r/198109/oisst-avhrr-v02r01.19810902.nc

Index({DimIndex(name='time', index=14370, sequence_len=14372, operation=<CombineOp.CONCAT: 
2>)}) https://www.ncei.noaa.gov/data/sea-surface-temperature-optimum-interpolation/v2.1/acces
s/avhrr/202101/oisst-avhrr-v02r01.20210104.nc

Index({DimIndex(name='time', index=14371, sequence_len=14372, operation=<CombineOp.CONCAT: 
2>)}) https://www.ncei.noaa.gov/data/sea-surface-temperature-optimum-interpolation/v2.1/acces
s/avhrr/202101/oisst-avhrr-v02r01.20210105.nc

<img src='architecture.png'>

In [6]:
import s3fs
import xarray as xr

endpoint_url = "https://ncsa.osn.xsede.org"
fs_osn = s3fs.S3FileSystem(anon=True, client_kwargs={"endpoint_url": endpoint_url},)

path = "s3://Pangeo/pangeo-forge/noaa_oisst/v2.1-avhrr.zarr"
ds = xr.open_zarr(fs_osn.get_mapper(path), consolidated=True)
print(f"{ds.nbytes/1e9:.2f} GB")
print(ds)

241.07 GB

<xarray.Dataset>
Dimensions:  (time: 14532, zlev: 1, lat: 720, lon: 1440)
Coordinates:
  * lat      (lat) float32 -89.88 -89.62 -89.38 -89.12 ... 89.38 89.62 89.88
  * lon      (lon) float32 0.125 0.375 0.625 0.875 ... 359.1 359.4 359.6 359.9
  * time     (time) datetime64[ns] 1981-09-01T12:00:00 ... 2021-06-14T12:00:00
  * zlev     (zlev) float32 0.0
Data variables:
    anom     (time, zlev, lat, lon) float32 dask.array<chunksize=(20, 1, 720, 1440), 
meta=np.ndarray>
    err      (time, zlev, lat, lon) float32 dask.array<chunksize=(20, 1, 720, 1440), 
meta=np.ndarray>
    ice      (time, zlev, lat, lon) float32 dask.array<chunksize=(20, 1, 720, 1440), 
meta=np.ndarray>
    sst      (time, zlev, lat, lon) float32 dask.array<chunksize=(20, 1, 720, 1440), 
meta=np.ndarray>
Attributes: (12/37)
    Conventions:                CF-1.6, ACDD-1.3
    cdm_data_type:              Grid
    comment:                    Data was converted from NetCDF-3 to NetCDF-4 ...
    creator_email:              oisst-help@noaa.gov
    creator_url:                https://www.ncei.noaa.gov/
    date_created:               2020-05-08T19:05:13Z
    ...                         ...
    source:                     ICOADS, NCEP_GTS, GSFC_ICE, NCEP_ICE, Pathfin...
    standard_name_vocabulary:   CF Standard Name Table (v40, 25 January 2017)
    summary:                    NOAAs 1/4-degree Daily Optimum Interpolation ...
    time_coverage_end:          1981-09-01T23:59:59Z
    time_coverage_start:        1981-09-01T00:00:00Z
    title:                      NOAA/NCEI 1/4 Degree Daily Optimum Interpolat...

<center>
<img src="pangeo-forge-logo-blue-2.png">

<h1> https://pangeo-forge.readthedocs.io/ </h1>
</center>